In [113]:
import os
from sagemaker.local import LocalSession

sagemaker_session = LocalSession()
sagemaker_session.config = {'local': {'local_code': True}}

os.environ["AWS_DEFAULT_REGION"] = "us-east-1"

# Set Up docker and .tar.gz

In [114]:
%%bash
cd .. && ./build.sh ./Dockerfile

building docker container with name transform_job
Sending build context to Docker daemon  14.34kB
Step 1/17 : FROM ubuntu:18.04
 ---> 5d2df19066ac
Step 2/17 : LABEL com.amazonaws.sagemaker.capabilities.multi-models=true
 ---> Using cache
 ---> 8ac2d20bd209
Step 3/17 : LABEL com.amazonaws.sagemaker.capabilities.accept-bind-to-port=true
 ---> Using cache
 ---> b7eaa0f56164
Step 4/17 : RUN apt-get update && apt-get upgrade -y && apt-get clean
 ---> Using cache
 ---> 8c5bb3da17a4
Step 5/17 : RUN apt-get install -y curl python3.7 python3.7-dev python3.7-distutils
 ---> Using cache
 ---> e762fbc99c8c
Step 6/17 : RUN update-alternatives --install /usr/bin/python python /usr/bin/python3.7 1
 ---> Using cache
 ---> 7998b6bb7638
Step 7/17 : RUN update-alternatives --set python /usr/bin/python3.7
 ---> Using cache
 ---> 3ec8e511d35c
Step 8/17 : RUN apt-get -y install --no-install-recommends     build-essential     ca-certificates     openjdk-8-jdk-headless     curl     vim     && rm -rf /var/lib/

In [115]:
%%bash
cd .. && tar -czvf artifact.tar.gz code/* model.json && tar -ztvf artifact.tar.gz

code/inference.py
code/requirements.txt
model.json
-rw-rw-r-- manu-lasker26/manu-lasker26 582 2023-01-31 23:06 code/inference.py
-rw-rw-r-- manu-lasker26/manu-lasker26   6 2023-01-31 21:59 code/requirements.txt
-rw-rw-r-- manu-lasker26/manu-lasker26  30 2023-01-31 21:36 model.json


# Set up local Transform job and model

In [116]:
directory = os.path.join(*os.getcwd().split(os.sep)[:-1])
directory

'home/manu-lasker26/Documents/manulasker/personal_projects/sagemaker-transform-job-inference/transform-job'

In [117]:
from sagemaker.transformer import Transformer
from sagemaker.model import Model

env_variables = {
    "SAGEMAKER_PROGRAM": "inference.py"
}

model = Model(
    image_uri = "transform_job",
    model_data = f"file:///{directory}/artifact.tar.gz",
    role = "arn:aws:iam::12345678910:role/test_role",
    env = env_variables,
    name = "test_model",
    sagemaker_session = sagemaker_session
)

In [118]:
model.create()

INFO:sagemaker:Creating model with name: test_model


In [119]:
transform = Transformer(
    model_name = model.name,
    instance_count = 1,
    instance_type = "local",
    output_path = f"file:///{directory}/output",
    sagemaker_session = sagemaker_session,
    env = env_variables
)

In [120]:
transform.transform(data = f"file:///{directory}/data/", content_type="application/json")

INFO:sagemaker:Creating transform job with name: transform_job-2023-02-01-04-16-20-678
INFO:sagemaker.local.image:serving
INFO:sagemaker.local.image:creating hosting dir in /tmp/tmpjhxr1evh
INFO:botocore.credentials:Found credentials in environment variables.
INFO:sagemaker.local.image:docker compose file: 
networks:
  sagemaker-local:
    name: sagemaker-local
services:
  algo-1-lw9gs:
    command: serve
    container_name: 4hym5pj681-algo-1-lw9gs
    environment:
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    image: transform_job
    networks:
      sagemaker-local:
        aliases:
        - algo-1-lw9gs
    ports:
    - 8080:8080
    stdin_open: true
    tty: true
    volumes:
    - /home/manu-lasker26/Documents/manulasker/personal_projects/sagemaker-transform-job-inference/transform-job:/opt/ml/model
version: '2.3'

INFO:sagemaker.local.image:docker command: docker-compose -f /tmp/tmpjhxr1evh/docker-compose.yaml up --build

Attaching to 4hym5pj681-algo-1-lw9gs
4hym5pj681-algo-1-lw9gs | Collecting pandas
4hym5pj681-algo-1-lw9gs |   Downloading pandas-1.3.5-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 33.1 MB/s eta 0:00:0000:0100:01:--:--
4hym5pj681-algo-1-lw9gs | Requirement already satisfied: numpy>=1.17.3 in /usr/local/lib/python3.7/dist-packages (from pandas->-r /opt/ml/model/code/requirements.txt (line 1)) (1.21.6)
4hym5pj681-algo-1-lw9gs | Collecting python-dateutil>=2.7.3
4hym5pj681-algo-1-lw9gs |   Downloading python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 kB 33.1 MB/s eta 0:00:0031m? eta -:--:--
4hym5pj681-algo-1-lw9gs | Collecting pytz>=2017.3
4hym5pj681-algo-1-lw9gs |   Downloading pytz-2022.7.1-py2.py3-none-any.whl (499 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.4/499.4 kB 44.4 MB/s eta 0:00:0031m? eta -:--:--
4hym5pj681-algo-1-lw9gs | Requiremen

INFO:sagemaker.local.entities:Checking if serving container is up, attempt: 10
INFO:sagemaker.local.entities:Container still not up, got: -1


4hym5pj681-algo-1-lw9gs | Successfully installed pandas-1.3.5 python-dateutil-2.8.2 pytz-2022.7.1
4hym5pj681-algo-1-lw9gs | WARNING: Running pip as the 'root' user can result in broken permissions and conflicting behaviour with the system package manager. It is recommended to use a virtual environment instead: https://pip.pypa.io/warnings/venv
4hym5pj681-algo-1-lw9gs | Warning: MMS is using non-default JVM parameters: -XX:-UseContainerSupport
4hym5pj681-algo-1-lw9gs | 2023-02-01T04:16:30,222 [INFO ] main com.amazonaws.ml.mms.ModelServer - 
4hym5pj681-algo-1-lw9gs | MMS Home: /usr/local/lib/python3.7/dist-packages
4hym5pj681-algo-1-lw9gs | Current directory: /
4hym5pj681-algo-1-lw9gs | Temp directory: /tmp
4hym5pj681-algo-1-lw9gs | Number of GPUs: 0
4hym5pj681-algo-1-lw9gs | Number of CPUs: 8
4hym5pj681-algo-1-lw9gs | Max heap size: 3525 M
4hym5pj681-algo-1-lw9gs | Python executable: /usr/bin/python
4hym5pj681-algo-1-lw9gs | Config file: /etc/sagemaker-mms.properties
4hym5pj681-algo-1-l

INFO:sagemaker.local.entities:Checking if serving container is up, attempt: 15
INFO:root:copying /tmp/tmpersmgka5/data_1.json.out -> /home/manu-lasker26/Documents/manulasker/personal_projects/sagemaker-transform-job-inference/transform-job/output


4hym5pj681-algo-1-lw9gs | 2023-02-01T04:16:32,815 [INFO ] pool-2-thread-10 ACCESS_LOG - /172.18.0.1:53458 "GET /ping HTTP/1.1" 200 7
4hym5pj681-algo-1-lw9gs | 2023-02-01T04:16:32,852 [INFO ] epollEventLoopGroup-3-2 ACCESS_LOG - /172.18.0.1:53466 "GET /execution-parameters HTTP/1.1" 404 1
4hym5pj681-algo-1-lw9gs | 2023-02-01T04:16:32,891 [INFO ] W-9000-model com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 1
4hym5pj681-algo-1-lw9gs | 2023-02-01T04:16:32,891 [INFO ] W-model-4-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - {'data_number': '1'}
4hym5pj681-algo-1-lw9gs | 2023-02-01T04:16:32,891 [WARN ] W-model-4-stderr com.amazonaws.ml.mms.wlm.WorkerLifeCycle - 2023-02-01 04:16:32,890 - sagemaker-inference - INFO - {'data_number': '1'}
4hym5pj681-algo-1-lw9gs | 2023-02-01T04:16:32,891 [INFO ] W-model-4-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - {'model': 'hola mundo'}
4hym5pj681-algo-1-lw9gs | 2023-02-01T04:16:32,891 [WARN ] W-model-4-stderr com.amazonaws.ml.mms.wlm.

# Results

In [104]:
import json
import os

out_files = [file for file in os.listdir(f"/{directory}/output/") if file.endswith("out")]

for out_file in out_files:
    with open(f"/{directory}/output/{out_file}", "r") as file:
        print(json.dumps(json.load(file)))

{"data_number": "1"}


# Clean Test

In [121]:
%%bash

cd .. && rm artifact.tar.gz && rm -r output/*.out